In [17]:
import geopandas as gpd
import pandas as pd
import folium
from folium import plugins
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

random.seed(42)

In [106]:
gdf = gpd.read_file("AL_Municipios_2024/AL_Municipios_2024.shp")
gdf["CD_MUN"] = gdf["CD_MUN"].astype(int)
display(gdf.head())
gdf.info()


,CD_MUN,NM_MUN,CD_RGI,NM_RGI,CD_RGINT,NM_RGINT,CD_UF,NM_UF,SIGLA_UF,CD_REGIA,NM_REGIA,SIGLA_RG,CD_CONCU,NM_CONCU,AREA_KM2,geometry
0,2708600,São Miguel dos Campos,270004,São Miguel dos Campos,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,335.679,"POLYGON ((-36.0739 -9.70094, -36.07339 -9.7008..."
1,2701407,Campo Alegre,270004,São Miguel dos Campos,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,312.726,"POLYGON ((-36.313 -9.88693, -36.31312 -9.88691..."
2,2700102,Água Branca,270009,Delmiro Gouveia,2702,Arapiraca,27,Alagoas,AL,2,Nordeste,NE,None,None,473.845,"POLYGON ((-37.93395 -9.37265, -37.97968 -9.347..."
3,2703205,Igreja Nova,270003,Penedo,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,428.023,"POLYGON ((-36.72973 -10.18535, -36.72974 -10.1..."
4,2709103,Taquarana,270007,Arapiraca,2702,Arapiraca,27,Alagoas,AL,2,Nordeste,NE,None,None,153.833,"POLYGON ((-36.48281 -9.6053, -36.4828 -9.60533..."


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   CD_MUN    102 non-null    int64   
 1   NM_MUN    102 non-null    object  
 2   CD_RGI    102 non-null    object  
 3   NM_RGI    102 non-null    object  
 4   CD_RGINT  102 non-null    object  
 5   NM_RGINT  102 non-null    object  
 6   CD_UF     102 non-null    object  
 7   NM_UF     102 non-null    object  
 8   SIGLA_UF  102 non-null    object  
 9   CD_REGIA  102 non-null    object  
 10  NM_REGIA  102 non-null    object  
 11  SIGLA_RG  102 non-null    object  
 12  CD_CONCU  10 non-null     object  
 13  NM_CONCU  10 non-null     object  
 14  AREA_KM2  102 non-null    float64 
 15  geometry  102 non-null    geometry
dtypes: float64(1), geometry(1), int64(1), object(13)
memory usage: 12.9+ KB


In [107]:
def criar_distribuicao_aleatoria(municipios):
    distribuicao_acompanhadas = {}
    distribuicao_alerta = {}
    distribuicao_critico = {}
    
    for municipio in municipios:
        # Crianças acompanhadas (0-5, 30% chance de ser 0)
        rand = random.random()
        if rand < 0.3:
            acompanhadas = 0
        else:
            acompanhadas = random.randint(1, 5)
        distribuicao_acompanhadas[municipio] = acompanhadas
        
        # Alertas (0-3, baseado no número de acompanhadas)
        if acompanhadas == 0:
            alertas = 0
        else:
            # Máximo de alertas não pode ser maior que acompanhadas
            max_alertas = min(3, acompanhadas)
            alertas = random.randint(0, max_alertas)
        distribuicao_alerta[municipio] = alertas
        
        # Críticos (0-2, baseado no número de acompanhadas)
        if acompanhadas == 0:
            criticos = 0
        else:
            # Máximo de críticos não pode ser maior que acompanhadas
            max_criticos = min(2, acompanhadas)
            criticos = random.randint(0, max_criticos)
        distribuicao_critico[municipio] = criticos
    
    return distribuicao_acompanhadas, distribuicao_alerta, distribuicao_critico

In [108]:
distribuicao_acompanhadas, distribuicao_alerta, distribuicao_critico = criar_distribuicao_aleatoria(gdf["NM_MUN"])

distribuicao_acompanhadas["Maceió"] = 42
distribuicao_acompanhadas["Arapiraca"] = 17
print(f"\nEstatísticas da distribuição:")
total_criancas = sum(distribuicao_acompanhadas.values())
municipios_com_zero = sum(1 for v in distribuicao_acompanhadas.values() if v == 0)
municipios_com_casos = len(distribuicao_acompanhadas) - municipios_com_zero

print(f"Total de crianças: {total_criancas}")
print(f"Municípios com 0 casos: {municipios_com_zero} ({municipios_com_zero/102*100:.1f}%)")
print(f"Municípios com casos: {municipios_com_casos} ({municipios_com_casos/102*100:.1f}%)")

distribuicao_alerta["Maceió"] = 15
distribuicao_alerta["Arapiraca"] = 8

distribuicao_critico["Maceió"] = 7
distribuicao_critico["Arapiraca"] = 4


Estatísticas da distribuição:
Total de crianças: 305
Municípios com 0 casos: 21 (20.6%)
Municípios com casos: 81 (79.4%)


In [109]:
gdf['RN acompanhados'] = gdf['NM_MUN'].map(distribuicao_acompanhadas)
gdf['RN em alerta'] = gdf['NM_MUN'].map(distribuicao_alerta)
gdf['RN criticos'] = gdf['NM_MUN'].map(distribuicao_critico)
gdf

,CD_MUN,NM_MUN,CD_RGI,NM_RGI,CD_RGINT,NM_RGINT,CD_UF,NM_UF,SIGLA_UF,CD_REGIA,NM_REGIA,SIGLA_RG,CD_CONCU,NM_CONCU,AREA_KM2,geometry,RN acompanhados,RN em alerta,RN criticos
0,2708600,São Miguel dos Campos,270004,São Miguel dos Campos,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,335.679,"POLYGON ((-36.0739 -9.70094, -36.07339 -9.7008...",0,0,0
1,2701407,Campo Alegre,270004,São Miguel dos Campos,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,312.726,"POLYGON ((-36.313 -9.88693, -36.31312 -9.88691...",2,0,2
2,2700102,Água Branca,270009,Delmiro Gouveia,2702,Arapiraca,27,Alagoas,AL,2,Nordeste,NE,None,None,473.845,"POLYGON ((-37.93395 -9.37265, -37.97968 -9.347...",4,0,0
3,2703205,Igreja Nova,270003,Penedo,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,428.023,"POLYGON ((-36.72973 -10.18535, -36.72974 -10.1...",4,3,1
4,2709103,Taquarana,270007,Arapiraca,2702,Arapiraca,27,Alagoas,AL,2,Nordeste,NE,None,None,153.833,"POLYGON ((-36.48281 -9.6053, -36.4828 -9.60533...",4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2704401,Major Isidoro,270008,Palmeira dos Índios,2702,Arapiraca,27,Alagoas,AL,2,Nordeste,NE,None,None,437.603,"POLYGON ((-37.112 -9.60618, -37.11246 -9.6061,...",4,1,1
98,2700904,Belo Monte,270011,Pão de Açúcar - Olho d'Água das Flores - Batalha,2702,Arapiraca,27,Alagoas,AL,2,Nordeste,NE,None,None,322.887,"POLYGON ((-37.0455 -9.80704, -37.04531 -9.8069...",3,2,1
99,2702702,Feliz Deserto,270003,Penedo,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,110.062,"POLYGON ((-36.32011 -10.3432, -36.32483 -10.33...",2,1,2
100,2707503,Porto Real do Colégio,270003,Penedo,2701,Maceió,27,Alagoas,AL,2,Nordeste,NE,None,None,237.017,"POLYGON ((-36.67182 -10.07628, -36.67653 -10.1...",5,2,2


In [110]:
def criar_mapa(coluna, color):
    # Calcular centróide de Alagoas para centralizar o mapa
    bounds = gdf.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Criar mapa base
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=8,
        tiles='OpenStreetMap'
    )
    
    # Definir escala de cores baseada no número de crianças
    max_criancas = gdf[coluna].max()
    
    # Usar paleta de cores Reds do matplotlib
    cmap = getattr(plt.cm, color)
    
    # Função para definir cor usando a paleta Reds
    def get_color(num_criancas):
        if num_criancas == 0:
            return '#ffffff'  # Branco para 0 casos
        else:
            # Normalizar valor entre 0.2 e 1.0 para evitar cores muito claras
            normalized_value = 0.2 + (num_criancas / 8) * 0.8
            rgba = cmap(normalized_value)
            # Converter RGBA para hexadecimal
            return mcolors.rgb2hex(rgba[:3])
    
    # Adicionar camada dos municípios
    folium.GeoJson(
        gdf,
        style_function=lambda feature: {
            'fillColor': get_color(feature['properties'][coluna]),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
            'opacity': 1
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['NM_MUN', coluna],
            aliases=['Município:', coluna],
            style="background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"
        ),
        popup=folium.GeoJsonPopup(
            fields=['NM_MUN', coluna],
            aliases=['Município:', coluna],
            style="background-color: yellow;",
        )
    ).add_to(m)
    
    
    return m

In [111]:
mapa_criancas = criar_mapa("RN acompanhados", "viridis")

# Salvar o mapa
mapa_criancas.save('./maps/mapa_RN_acompanhados.html')


In [112]:
mapa_criancas = criar_mapa("RN em alerta", "Oranges")

# Salvar o mapa
mapa_criancas.save('./maps/mapa_RN_alerta.html')

In [113]:
mapa_criancas = criar_mapa("RN criticos", "Reds")

# Salvar o mapa
mapa_criancas.save('./maps/mapa_RN_criticos.html')

In [114]:
mapa_criancas = criar_mapa("RN acompanhados", "Blues")

# Salvar o mapa
mapa_criancas.save('./maps/mapa_RN_taxa_sucesso.html')